On Wednesday, 6 March, Chancellor of the Exchequer Jeremy Hunt will present the government's 2024 Spring Budget. Though the specific budgetary provisions Number 11 will put forth remain private, various sources speculate that the chancellor will cut the National Insurance rate, a supplement to his previous cuts in autumn 2023 (analysed by PolicyEngine in [this article](https://policyengine.org/uk/research/autumn-statement-2023)).

In this post, we'll estimate the impacts of such a cut upon the budget, as well as upon society-wide economic performance indicators. We'll also look at the impact these changes would have upon an average UK household.

# Reforms within Spring Budget 2024

News media speculation regarding the Spring Budget's proposed reforms has mooted changes to various different tax provisions. In this article, we will utilise PolicyEngine to model and analyse three:
* Cuts to Employee (Class 1) National Insurance Contributions (NICs) by either 1p[^1] or 2p[^3]
* A 2p cut to the basic rate of tax, lowering the rate from 20p to 18p[^4]
* A fuel duty freeze[^1]

In modelling these reforms, we will assume each took effect on 1 January 2024 and will evaluate their impacts over calendar year 2024. In the future, PolicyEngine hopes to develop the capability to accurately model the true effective date of these sorts of policies.


[^1]:
      [The Guardian, 2024](https://www.theguardian.com/politics/2024/mar/02/tax-and-spending-cuts-will-backfire-economists-warn-jeremy-hunt)
[^2]: 
      [Institute for Government, 2024](https://www.instituteforgovernment.org.uk/comment/five-things-look-out-spring-budget-2024)
[^3]:
      [Financial Times, 2024](https://www.ft.com/content/c2d56d45-c061-4d5a-bf4f-1eddd1fa7d25)
[^4]:
      [Financial Times, 2024](https://www.ft.com/content/1b4f19d7-c7e5-4d16-a7c1-1c12ec0d0592)

In [1]:
!pip install dpath

zsh:1: permission denied: pip


In [2]:
# Code producing budgetary impact table.

from policyengine_uk.model_api import *
from policyengine_uk import Microsimulation
import pandas as pd

# DataFrame with columns: [provision name, provision cost in 2024, percent better off, provision poverty impact in 2024, inequality impact in 2024]

reforms = []

ni_rate_cut_1p = {
    "gov.hmrc.national_insurance.class_1.rates.employee.main": {
        "year:2024:5": 0.09,
    }
}

ni_rate_cut_2p = {
    "gov.hmrc.national_insurance.class_1.rates.employee.main": {
        "year:2024:5": 0.08,
    }
}

base_rate_cut_2p = {
    "gov.hmrc.income_tax.rates.uk[0].rate": {
        "year:2024:5": 0.18,
    }
}

fuel_duty_freeze = {
    "gov.hmrc.fuel_duty.petrol_and_diesel": {
        "year:2024:5": 0.5083
    }
}

# Space for others

reforms = [ni_rate_cut_1p, ...]

provisions = ["1p NI cut", "2p NI cut", "2p basic rate cut", "Fuel duty cut"]

baseline = Microsimulation()

def get_impacts(reform_dict: dict) -> dict:
    reform = Reform.from_dict(reform_dict)
    reformed = Microsimulation(reform=reform)

    budgetary_impact = baseline.calculate("household_net_income", 2024).sum() - reformed.calculate("household_net_income", 2024).sum()

    return dict(
        budgetary_impact=budgetary_impact,
        reform_id=reform.api_id,
    )

budgetary_impacts = []
reform_ids = []

for i in range(len(provisions)):
    impacts = get_impacts(reforms[i])
    budgetary_impacts.append(impacts["budgetary_impact"])
    reform_ids.append(impacts["reform_id"])

df = pd.DataFrame({
    "Reform": provisions,
    "Budgetary impact": budgetary_impacts,
    "PolicyEngine": reform_ids
})

df["Budgetary impact"] = df["Budgetary impact"].apply(lambda x: f"{x:,.1f}")
df["PolicyEngine"] = df["PolicyEngine"].apply(lambda x: f"#{x}")

ModuleNotFoundError: No module named 'policyengine_uk'


# NIC Cuts
Turning first to the macro impacts, the PolicyEngine model projects that a reduction in the Class 1 NIC contribution rate by 2p costs the government approximately £10.8 billion.

[chart]

Under such a reform, every income decile stands to gain financially, with 63p of households increasing their post-tax income, and none seeing a decrease in take-home pay. However, top earners do especially well. With a reduction by 2p in Class 1 NIC contribution rates, the average household in income decile 1 receives £4 more post-tax income, while the average household in decile 10 receives £1,087. Under a reduction by 1p, these values are halved, to £2 and £585, respectively.

[chart]

This means that the NIC contribution reform generates an ambiguous effect upon income inequality - while the top 1% of the population's share of wealth decreases, the GINI income inequality coefficient actually increases by 0.4p. The poverty rate would also be projected to decrease, but quite modestly (0.1 percentage points, or 0.3p against its current rate).

[charts]

Turning to the household level, reducing NIC contributions by 2p increases post-tax income for a single person with employment income if they earn at least £12,570, much like the reforms previously modelled by PolicyEngine following the Autumn Statement 2023. Said individual's take-home pay would increase linearly until reaching a maximum benefit of £817 at an annual income of £50,500. Were NIC contributions to only be reduced by 1p, the income phase-in and phase-out points remain the same, but said individual's maximum benefit would be only £440 annually.

Mirroring the reforms from autumn 2023, most individuals within the proscribed income range would see each 1p reduction in NIC contributions correspond with a 1p reduction in marginal tax rate. Individuals on the very ends of the range would see slightly lower reductions. [why is this?]

[charts]

# Basic Rate Cuts




In [ ]:
import pandas as pd
import plotly.express as px
from policyengine_core.charts import *
from policyengine_uk import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
from plotly.express.colors import sample_colorscale


def modify_parameters(parameters):
    parameters.gov.hmrc.national_insurance.class_1.rates.employee.main.update(
        start=instant("2024-01-01"), stop=instant("2024-12-31"),
        value=0.08)
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
baseline_person = baseline.calc("household_net_income",
    period=2024, map_to="person")
reformed_person = reformed.calc("household_net_income",
    period=2024, map_to="person")
difference_person = reformed_person - baseline_person


# fig = format_fig(fig)
# 
# for i in range(len(fig.data)):
#     fig.data[i].marker.color = c[i]
# 
# fig.update_layout(
#     legend_title="Calendar year",
#     xaxis_tickvals=chosen_metrics,
#     xaxis_ticktext=["Child Benefit", "Housing Benefit", "Universal Credit"],
#     xaxis_title="",
#     yaxis_title="Budgetary impact (£)",
# )

ModuleNotFoundError: No module named 'pandas'